<a href="https://colab.research.google.com/github/zerotodeeplearning/ztdl-masterclasses/blob/master/solutions_do_not_open/Transfer_Learning_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transfer Learning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# sports_images_path = tf.keras.utils.get_file(
#     'sports_images',
#     'https://archive.org/download/ztdl_sports_images/sports_images.tgz',
#      untar=True)

In [ ]:
![[ ! -f sports_images.tar.gz ]] && gsutil cp gs://ztdl-datasets/sports_images.tar.gz .
![[ ! -d sports_images ]] && echo "Extracting images..." && tar zxf sports_images.tar.gz
sports_images_path  = './sports_images'

In [ ]:
train_path = os.path.join(sports_images_path, 'train')
test_path = os.path.join(sports_images_path, 'test')

In [ ]:
batch_size = 32
img_size = 299

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
base_model = Xception(include_top=False,
                      weights='imagenet',
                      input_shape=(img_size, img_size, 3),
                      pooling='avg')

In [ ]:
model = Sequential([
    base_model,
    Dense(20, activation='softmax')
])

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode = 'sparse',
    shuffle=True)

In [ ]:
model.fit(train_generator,
          epochs=1,
          steps_per_epoch=len(train_generator))

### Exercise 1:

Improve the model above

- Insert a few more layers between the `base_model` and the output `Dense` layer. You could include:
    - Additional `Dense` layers
    - `Dropout` layers
    - `BatchNormalization` layers
    
    ```python
    model = Sequential([
        base_model,
        # YOUR CODE HERE
        Dense(20, activation='softmax')
    ])
    ```
- Compile the model
- Freeze the `base_model` weights
- Fit the model on the `train_generator`

- Define a new `test_generator` using the `test_path` and a `batch_size` of 500 and use it to create test batch:
    ```python
    test_generator = datagen.flow_from_directory(
        # YOUR CODE HERE
    )
    
    X_test, y_test = test_generator.next()
    ```
- Generate predictions on the test batch (these will be probabilities) and use the `.argmax()` function to extract the predicted classes
- Evaluate your predictions using the `classification_report` and `confusion_matrix`

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
model = Sequential([
    base_model,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(20, activation='softmax')
])

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.layers[0].trainable=False

In [ ]:
model.fit(train_generator,
          epochs=2,
          steps_per_epoch=len(train_generator))

In [ ]:
test_generator = datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=500,
    class_mode = 'sparse',
    shuffle=True)

In [ ]:
X_test, y_test = test_generator.next()

In [ ]:
y_test_pred_proba = model.predict(X_test)

In [ ]:
y_test_pred = y_test_pred_proba.argmax(axis=1)

In [ ]:
y_test_pred_proba.shape

In [ ]:
classes_dict = test_generator.class_indices
classes = list(classes_dict.keys())

In [ ]:
print(classification_report(y_test, y_test_pred, target_names=classes))

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

pd.DataFrame(cm,
             index=classes,
             columns=classes).style.bar(color='lightgreen', vmin=0, vmax=50)

### Exercise 2: Tensorflow Hub

Pre-trained models can also be found on [TensorFlow Hub](https://www.tensorflow.org/hub)

They can be loaded by providing the appropriate link to the `hub.KerasLayer` layer.

- Load the model provided here: https://tfhub.dev/google/bit/m-r50x3/1 into a Keras layer, call it `base_model_2`
- Define a new transfer learning model like the previous one and check that it trains correctly on the train generator
- You may have to adapt the generator with the following preprocessing function: `lambda x:(x/127.5) - 1`

In [ ]:
import tensorflow_hub as hub

In [ ]:
base_model_2 = hub.KerasLayer("https://tfhub.dev/google/bit/m-r50x3/1")

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=lambda x:(x/127.5) - 1)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode = 'sparse',
    shuffle=True)

In [ ]:
model = Sequential([
  base_model_2,
  Dense(256, activation='relu'),
  Dropout(0.5),
  Dense(20, activation='softmax')
])

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.layers[0].trainable=False

In [ ]:
model.fit(train_generator,
          epochs=2,
          steps_per_epoch=len(train_generator))